In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!mkdir data
!cp "/content/drive/MyDrive/cleansed_product_images.npy" "data/cleansed_product_images.npy"

In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.1 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 6.6 MB 28.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import time
import copy
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
from google.colab import drive
from transformers import BertTokenizer
from transformers import BertModel
from tqdm import tqdm
import pickle


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')
products = np.load("data/cleansed_product_images.npy", allow_pickle=True)
products = pd.DataFrame(products, columns=['product_id', 'product_name', 'category', 'product_description', 'price', 'location', 'url', 'page_id', 'create_time', 'local_area', 'city', 'price_gbp', 'sub_cat_0', 'sub_cat_1', 'sub_cat_2', 'sub_cat_3', 'sub_cat_4', 'img_id', 'img_array'])
products.head(5)

Using cpu for inference


,product_id,product_name,category,product_description,price,location,url,page_id,create_time,local_area,city,price_gbp,sub_cat_0,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_4,img_id,img_array
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,Mirror wall art,"Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...,1426704584,2022-02-26,Wokingham,Berkshire,5.0,Home & Garden,"Dining, Living Room Furniture","Mirrors, Clocks & Ornaments",<NA>,<NA>,64aa79f3-e9fa-417c-a332-714b8ce933f1,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,Stainless Steel Food Steamer,Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26,Inverness,Highland,20.0,Home & Garden,Other Household Goods,<NA>,<NA>,<NA>,4e670f9e-7feb-458f-b529-ac52547abe2b,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
2,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,Stainless Steel Food Steamer,Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...,1426704579,2022-02-26,Inverness,Highland,20.0,Home & Garden,Other Household Goods,<NA>,<NA>,<NA>,a864ee52-d91e-46e7-94d1-2418e9bb2877,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,Sun loungers,Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...,1426704576,2022-02-26,Skegness,Lincolnshire,20.0,Home & Garden,Garden & Patio,Outdoor Settings & Furniture,<NA>,<NA>,bfe77c38-c9eb-47fb-b3d6-31ffdefb6ff9,"[[[29, 37, 40], [22, 30, 33], [22, 30, 33], [2..."
4,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...,1426704575,2022-02-26,Radstock,Somerset,115.0,Home & Garden,"Dining, Living Room Furniture",Other,<NA>,<NA>,a92e56b7-94fc-41b4-ba6c-f2f224f42bb8,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."


Create text dataset made up of tokenised text and numerical labels

In [5]:
class BertProductDescriptionCategoryDataset(Dataset):
    def __init__(self, data, input_pad_length=None, transform=None):
        super().__init__()
        self.labels = data['sub_cat_0'].to_list()
        self.num_classes = len(set(self.labels))
        self.descriptions = data['product_description']
        self.descriptions_list = self.descriptions.to_list()
        if input_pad_length is None:
          avg_description_length = sum( map(len, self.descriptions_list) ) / len(self.descriptions_list)
          self.pad_length = round(avg_description_length)
        else:
          self.pad_length = input_pad_length
        self.category_encoder = {y: x for (x, y) in enumerate(set(self.labels))}
        self.category_decoder = {x: y for (x, y) in enumerate(set(self.labels))}
        self.tokenised_descriptions = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
        assert len(self.descriptions_list) == len(self.labels)

    def __getitem__(self, index):
        label = int(self.category_encoder[self.labels[index]])
        label = torch.as_tensor(label).to(device)
        sentence = self.descriptions.iloc[index]
        encoded = self.tokenised_descriptions.batch_encode_plus([sentence], max_length=self.pad_length, padding='max_length', truncation=True)
        encoded = {key: torch.LongTensor(value) for key, value in encoded.items()}
        with torch.no_grad():
          features = self.model(**encoded).last_hidden_state.swapaxes(1,2)

        features = features.squeeze(0)

        return (features, label)

    def __len__(self):
        return len(self.descriptions_list)

In [6]:
class BertCNN(torch.nn.Module):
    def __init__(self, embedding_size=768, out_size=13):
        super(BertCNN, self).__init__()
        self.layers = torch.nn.Sequential(
            nn.Conv1d(embedding_size, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(256, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            # nn.Dropout(),
            nn.Flatten(),
            nn.Linear(1152 , 128),
            nn.ReLU(),
            nn.Linear(128, out_size)
            )

    def forward(self, X):
        X = self.layers(X)
        return X


In [8]:
dataset = BertProductDescriptionCategoryDataset(products, input_pad_length=292)
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)
criterion = nn.CrossEntropyLoss()
model = BertCNN().to(device)
optimiser = optim.Adam(model.parameters(), lr = 0.001)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
epochs = 5

for epoch in range(epochs):
    since = time.time()
    hist_accuracy = []
    hist_loss = []
    accuracy = 0
    pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    for i, batch in pbar:
        text_features, labels = batch
        text_features = text_features.to(device)
        labels = labels.to(device)
        optimiser.zero_grad()
        outputs = model(text_features)
        loss = criterion(outputs, labels)
        loss.backward()
        accuracy = torch.sum(torch.argmax(outputs, dim=1) == labels).item()/len(labels)
        hist_accuracy.append(accuracy)
        hist_loss.append(loss.item())
        optimiser.step()
        pbar.set_description(f"Epoch = {epoch+1}/{epochs}. loss = {loss.item():.4f} Acc = {round(torch.sum(torch.argmax(outputs, dim=1) == labels).item()/len(labels), 2)}, Total_acc = {round(np.mean(hist_accuracy), 2)}" )

time_elapsed = time.time() - since
print(time_elapsed)

torch.save(model.state_dict(), 'bert_model.pt')

with open('bert_decoder.pkl', 'wb') as f:
    pickle.dump(dataset.category_decoder, f)

Epoch = 1/5. loss = 2.5637 Acc = 0.06, Total_acc = 0.06:   0%|          | 1/346 [00:40<3:50:24, 40.07s/it]